In [5]:
import numpy as np
import pandas as pd
import math

from sklearn.datasets import make_classification

from sklearn.metrics import accuracy_score

from supervised import decision_tree


In [7]:
class RandomForest:
    """
    
    """
    def __init__(self, trees, n_trees, max_feature , min_sample_split, min_impurity, max_depth,
                 prediction_aggrigation_calculation):
        """
        """
        self.n_estimators = n_trees
        self.min_sample_split = min_sample_split
        self.min_impurity =min_impurity
        self.max_depth = max_depth
        self.max_features = max_feature
        self.tree_feature_indexes = []
        self.prediction_aggrigation_calculation = prediction_aggrigation_calculation 
        # Initialize the trees.
        self.trees = trees

    def _make_random_suset(self, X, y, n_subsets, replasment=True):
        """
        
        """
        subset = []
        # use 100% of data when replacement is true , use 50% otherwise.
        sample_size = np.shape[0] if replasment else (np.shape[0] // 2)

        # First concadinate the X and y datasets in order to make a choice.
        Xy = np.concatenate(X, y, axis=1)
        np.random.shuffle(Xy)
        for i in range(n_subsets):
            index = np.random.choice(range(sample_size), size=np.shape(range(sample_size)), replace=replasment)
            X = X[index][:, :-1]
            y = y[index][: , -1]
            subset.append([X, y])
        return subset

    def train(self, X, y):
        """
        """
        # if the max_features is not given then select it as square root of no on feature availabe.
        n_features = X.shape[1]
        if self.max_features == None:
            self.max_features = int(math.sqrt(n_features))

        # Split the dataset into number of subsets equal to n_estimators.
        subsets = self._make_random_suset(X, y, self.n_estimators)
        
        for i, subset in enumerate(subsets):
            X_subset , y_subset = subset[i][0], subset[i][1]

            # select a random sucset of features for each tree. This is called feature bagging.
            idx = np.random.choice(range(n_features), size=self.max_features, replace=True)
            # track this for prediction.
            self.tree_feature_indexes.append(idx)
            # Get the X with the selected features only.
            X_subset = X_subset[:, idx]

            # build the model with selected features and selected random subset from dataset.
            self.tree[i].train(X_subset, y_subset)

    def predict(self, test_X):
        """
        """
        # predict each sample one by one.
        y_preds = np.empty((test_X.shape[0], self.n_estimators))
        for i, tree in enumerate(self.trees):
            features_index = self.tree_feature_indexes[i]
            X_selected_features = test_X[:, features_index]
            y_preds[:, i] = tree.predict(X_selected_features)

        y_pred = self.prediction_aggrigation_calculation(y_preds)
        y_pred = []
        # for sample_prediction in y_preds:
        #     y_pred.append(np.bincount(sample_prediction.astype('int')).argmax())
        # y_preds = np.expand_dims(y_preds, axis=1)
        return y_pred

